# Implement a world clock with Flowpipe
This is very simple and in the end it is just a one liner in python, but a simple example like makes it easier to demonstrate the idea behind flowpipe.

Let's import the necessary classes:

In [1]:
from datetime import datetime
from time import time

from flowpipe.node import INode
from flowpipe.plug import OutputPlug, InputPlug
from flowpipe.graph import Graph
from flowpipe.engines.python_engine import PythonEngine

Then the desired functionality has to be implemented into Nodes.
We need two nodes, one to get the current time and one to convert it to a timezone.

In [2]:
class CurrentTime(INode):

    def __init__(self, name=None):
        super(CurrentTime, self).__init__(name)
        OutputPlug('time', self)

    def compute(self):
        return {'time': time()}

In [3]:
class ConvertTime(INode):

    def __init__(self, name=None, timezone=0):
        super(ConvertTime, self).__init__(name)
        InputPlug('time', self)
        InputPlug('timezone', self, timezone)
        OutputPlug('converted_time', self)

    def compute(self, time, timezone):
        return {'converted_time': [time + timezone * 60 * 60, self.name]}

The World Clock will receive the times and locations and display it.

In [4]:
class WorldClock(INode):

    def __init__(self, name=None):
        super(WorldClock, self).__init__(name)
        InputPlug('time1', self)
        InputPlug('time2', self)
        InputPlug('time3', self)

    def compute(self, time1, time2, time3):
        print('-- World Clock -------------------')
        print('It is now {0} in {1}'.format(
            datetime.fromtimestamp(time1[0]).strftime("%H:%M"), time1[1]))
        print('It is now {0} in {1}'.format(
            datetime.fromtimestamp(time2[0]).strftime("%H:%M"), time2[1]))
        print('It is now {0} in {1}'.format(
            datetime.fromtimestamp(time3[0]).strftime("%H:%M"), time3[1]))
        print('----------------------------------')

Now we can create the Graph that represents the world clock.
First create all the necessary Nodes.

In [5]:
current_time = CurrentTime()
van = ConvertTime(name='Vancouver', timezone=-8)
ldn = ConvertTime(name='London', timezone=0)
muc = ConvertTime(name='Munich', timezone=1)
world_clock = WorldClock()

Now, the Nodes can be wired together.
The bitshift operator is used as a shorthand to connect the plugs.

In [6]:
current_time.outputs['time'] >> van.inputs['time']
current_time.outputs['time'] >> ldn.inputs['time']
current_time.outputs['time'] >> muc.inputs['time']
van.outputs['converted_time'] >> world_clock.inputs['time1']
ldn.outputs['converted_time'] >> world_clock.inputs['time2']
muc.outputs['converted_time'] >> world_clock.inputs['time3']

The nodes are now grouped into a Graph, representing the world clock

In [7]:
graph = Graph(name="WorldClockGraph", nodes=[current_time, van, ldn, muc, world_clock])

The Graph can be visualized.

In [8]:
print(graph)

   0 | +-------------+          +----------------+          +------------+
   1 | | CurrentTime |          |   Vancouver    |          | WorldClock |
   2 | |-------------|          |----------------|          |------------|
   3 | |        time o-->--+    o timezone       |     +--->o time1      |
   4 | +-------------+     +--->o time           |     |--->o time2      |
   5 |                     |    | converted_time o-->--+--->o time3      |
   6 |                     |    +----------------+     |    +------------+
   7 |                     |    +----------------+     |                  
   8 |                     |    |     London     |     |                  
   9 |                     |    |----------------|     |                  
  10 |                     |    o timezone       |     |                  
  11 |                     +--->o time           |     |                  
  12 |                     |    | converted_time o-->--+                  
  13 |                   

The Graph is evaluated by an Engine. The PythonEngine will evaluate the Graph in the current python session.

In [9]:
engine = PythonEngine()
engine.evaluate_entire_graph(graph)

flowpipe DEBUG: Computed: CurrentTime
flowpipe DEBUG: Computed: Vancouver
flowpipe DEBUG: Computed: London
flowpipe DEBUG: Computed: Munich
flowpipe DEBUG: Computed: WorldClock
flowpipe DEBUG: Computed: WorldClockGraph


-- World Clock -------------------
It is now 14:55 in Vancouver
It is now 22:55 in London
It is now 23:55 in Munich
----------------------------------
